# Imports

In [13]:
import pandas as pd
import numpy as np
from datetime import datetime
import string

# Student Class

In [85]:
class Students:
    '''Inserts a student into our student table and controls student table.

    All students from all classes that the teacher teaches.
    '''
    students = pd.DataFrame(
        columns = [
            'id', 
            'first_name', 
            'last_name', 
            'gender', 
            'age'
        ]
    )
    # necessary for ID operations and consistency
    students.id = pd.to_numeric(students.id)

    student_id_generator = 1

    def insert_student(
            first_name: str,
            last_name: str,
            gender: str,
            age: int,
            student_id: int = None
    ) -> None:
        '''Inserts and updates students from the student table.
        '''
        first_name = first_name.lower()
        last_name = last_name.lower()
        gender = gender.lower()

        if not student_id:
            # really dumb way to make a unique student id
            while Students.student_id_generator in pd.to_numeric(Students.students.id.values):
                Students.student_id_generator += 1

            student_id = Students.student_id_generator
            Students.student_id_generator += 1

        # insert new row
        row = pd.DataFrame(data = [student_id, first_name, last_name, gender, age]).T
        row.columns = Students.students.columns
        Students.students = pd.concat([Students.students, row], ignore_index= True)

        # if student is already in table, delete old entry
        Students.students.drop_duplicates(
            subset= ['id'], 
            keep= 'last', 
            inplace= True,
            ignore_index= True
        )

    def delete_student(**kwargs) -> None:
        '''Deletes a student fromn the Students.students table

        Specify

        first_name: str,
        last_name: str

        OR

        student_id: int
        '''
        target_index = None
        if len(kwargs) == 0 or len(kwargs) >= 3:
            raise Exception(f'Expected argument of length 1 or 2. Found arguments of length:\t{len(kwargs)}')
        
        elif ('last_name' in kwargs) and ('first_name' in kwargs):
            target_index = Students.students.loc[
                (Students.students.first_name == kwargs['first_name']) & (Students.students.last_name == kwargs['last_name'])
            ].index[0]

        elif 'student_id' in kwargs:
            target_index = Students.students.loc[(Students.students.id == kwargs['student_id'])].index[0]

        else:
            raise Exception(
                'Invalid arguments passed. Expected arguments with either \
                    "first_name: str", "last_name: str" OR the singular argument "student_id: int"'
            )

        Students.students.drop(index= target_index, inplace= True)

# Comprehension Class

In [129]:
class Comprehension:
    '''Reading comprehension exam results and resources.
    '''
    exam_grades = pd.DataFrame(
        columns= [  # 1 indicates student passed a given question, 0 means fail
            'student_id', 
            'type',  # type of exam, informational or fictional
            'date',
            'incorrect',  # what the student failed
            'score'  # the sum of each row
        ]
    )

    convert_from_acronym = {
        'v': 'vocabulary',
        'kd': 'key_detail',
        'ca': 'character_analysis',  # exclsuive to Fictional exams, null else
        'ar': 'analyze_relationships',  # exclsuive to Informational exams, null else
        'i': 'infer',
        'e': 'evaluate',
        'l': 'literal',
        'tf': 'text_feature',  # exclsuive to Informational exams, null else
        'go': 'graphic_organizer',  # exclsuive to Fictional exams, null else
        'mi': 'main_idea'
    }

    fictional_only = ['ca', 'go']
    informational_only = ['ar', 'tf']
    base = list(convert_from_acronym.keys())
    fictional = base.copy()
    informational = base.copy()

    for elem in fictional_only + informational_only:
        base.remove(elem)
    for elem in fictional_only:
        informational.remove(elem)
    for elem in informational_only:
        fictional.remove(elem)

    base = frozenset(base)
    fictional = frozenset(fictional)
    informational = frozenset(informational)
    fictional_only = frozenset(fictional_only)
    informational_only = frozenset(informational_only)
    


    def insert_exam_grade(
            student_id: int,
            type_: str,
            incorrect: list,
            date: str = None,
            score: int = None
    ) -> None:
        '''Inserts and updates a students exam info into the exam_grades table.
        '''
        arg_converter = None

        # parse the args and verify min required args are present

        if any([True for item in incorrect if item in list(Comprehension.convert_from_acronym.values())]):
            arg_converter = {
                val: key for key,val in Comprehension.convert_from_acronym.items()
            }
            incorrect = [ arg_converter[elem] for elem in incorrect ]

        elif any([True for item in incorrect if item not in list(Comprehension.convert_from_acronym.keys())]):
            raise Exception(
                f'Missing necessary arguments. Expecting args similar to ...\
                    \n{Comprehension.convert_from_acronym}'
            )
        
        types = ['fictional', 'informational']
        if type_ not in types:
            raise Exception(f'Incorrect exam type found. Expected one of the following: {types}')

        # parse what can be conditional input
          
        if not date:
            date = datetime.today().strftime('%Y-%m-%d')

        if not score:
            score = 8 - len(incorrect)

        row = [
             student_id,
             type_,
             date,
             incorrect,
             score
        ]
        row = pd.DataFrame(data= row).T
        row.columns = Comprehension.exam_grades.columns
        Comprehension.exam_grades = pd.concat([Comprehension.exam_grades, row], ignore_index= True)
        
        # if student is already in table, delete old entry
        Comprehension.exam_grades.drop_duplicates(
            subset= ['student_id', 'date'], 
            keep= 'last', 
            inplace= True,
            ignore_index= True
        )

    def get_index_of_student(student_id: int, date: str = None) -> int:
        '''Finds the index in Comprehension.exam_grades where the student and date are specified

        If date = None, then finds most recent index.
        '''
        target_index = None

        if not date:
            target_index = Comprehension.exam_grades\
            .loc[Comprehension.exam_grades.student_id == student_id]\
            .sort_values(by= ['date'], axis= 0)\
            .index[-1]

        else:
            target_index = Comprehension.exam_grades.loc[
                (Comprehension.exam_grades.student_id == student_id)
                & (Comprehension.exam_grades.date == date)
            ].index[0]

        return target_index

    def delete_exam_grade(student_id: int, date: str = None) -> None:
        '''Deletes a student's exam grades from the grade book.

        If date = None, then deletes most recent instance.
        If date = "OVERRIDE_ALL", then deletes all instances.
        '''
        target_index = None

        if date == 'OVERRIDE_ALL':
            target_index = Comprehension.exam_grades.loc[
                (Comprehension.exam_grades.student_id == student_id)
            ].index

        else:
            target_index = Comprehension.get_index_of_student(student_id, date)

        Comprehension.exam_grades.drop(index= target_index, inplace= True)

    def get_student_response_breakdown(student_id: int, date: str = None) -> list:
        '''Returns what categories a student got wrong and right on a given exam.

        If date = None, most recent exam is selected.

        Returns tuple with first element being a list of correct topics and second being incorrect topics
        '''
        target_index = Comprehension.get_index_of_student(student_id, date)
        incorrect = set(Comprehension.exam_grades.iloc[target_index, 3][0])
        type_ = Comprehension.exam_grades.iloc[target_index, 1][0]

        correct = None
        if type_ == 'fictional':
            correct = Comprehension.fictional.difference(incorrect)
        else:
            correct = Comprehension.informational.difference(incorrect)

        return (list(correct), list(incorrect))
        

# Spelling Class

In [159]:
class Spelling:
    '''Spelling exam results and resources.
    '''
    stages = {
        'phonetic': {
            'short_vowels': frozenset([
                'o', 'u', 'e', 'o', 'a'
            ]),
            'consonant_blends': frozenset([
                'gr', '-nk', 'cl', '-st', 'sl', 'fl', 'spr', 'fr', 'tr', 'dr'
            ]),
            'consonant_digraphs': frozenset([
                'sh', 'th', 'ch', 'ck'
            ])
        },
        'transitional': {
            'long_vowels': frozenset([
                'o_e', 'ea', 'igh', 'oa', 'ai'
            ]),
            'complex_vowels': frozenset([
                'ou', 'or', 'ir', 'ur', 'aw', 'oo', 'oi'
            ])
        },
        'fluent': {
            'inflectional_endings': frozenset([
                '-edt /t/', '-ing (double consonant)', '-s (spelling change)',
                '-ing (spelling change)', 'ed'
            ]),
            'multisyllabic_words_2_syllabes': frozenset([
                '-ness (suffix)', 'mar (r-controlled syllable)', 'ble (end syllable)',
                '-ful (suffix)', '-tion (suffix)'
            ])
        },
        'advanced': {
            'multisyllabic_words_3_syllabes': frozenset([
                '-ure', '-ous', '-ity', '-able', '-ate'
            ])
        }
    }

    exam_grades = pd.DataFrame(
        columns= [
            'student_id', 
            'date',  #  below is what the student failed in this section
            'phonetic_short_vowels',
            'phonetic_consonant_blends',
            'phonetic_consonant_digraphs',
            'transitional_long_vowels',
            'transitional_complex_vowels',
            'fluent_inflectional_endings', 
            'fluent_multisyllabic_words_2_syllabes',
            'advanced_multisyllabic_words_3_syllabes',
            'score'  # the determined spelling stage of the student
        ]
    )

    def calculate_spelling_level(*incorrect_exam_results) -> str:
        '''Calculates the spelling level of a student based on test results.

        Expecting a list for each of the following and in the following order:

        phonetic_short_vowels: list,
        phonetic_consonant_blends: list,
        phonetic_consonant_digraphs: list,
        transitional_long_vowels: list,
        transitional_complex_vowels: list,
        fluent_inflectional_endings: list,
        fluent_multisyllabic_words_2_syllabes: list,
        advanced_multisyllabic_words_3_syllabes: list

        Returns the calculated spelling level as one of the following:
        "phonetic", "transitional", "fluent", "advanced"
        '''
        spelling_level = 'advanced'
        for i in range(len(incorrect_exam_results)):
            if len(incorrect_exam_results[i]) >= 2:

                if i in [0, 1, 3]:
                    spelling_level = 'phonetic'

                elif i in [4, 5]:
                    spelling_level = 'transitional'

                elif i in [6, 7]:
                    spelling_level = 'fluent'

                elif i in [8]:
                    spelling_level = 'advanced'

                else:
                    raise Exception(
                        f'Too many args passed in. Expected a list of 8 lists. Recieved a list of {len(incorrect_exam_results)} items'
                    )
                
                break
        
        return spelling_level
        
    def insert_exam_grade(
            student_id: int,
            phonetic_short_vowels: list,
            phonetic_consonant_blends: list,
            phonetic_consonant_digraphs: list,
            transitional_long_vowels: list,
            transitional_complex_vowels: list,
            fluent_inflectional_endings: list,
            fluent_multisyllabic_words_2_syllabes: list,
            advanced_multisyllabic_words_3_syllabes: list,
            date: str = None,
            score: str = None
    ) -> None:
        '''Inserts and updates a students exam info into the exam_grades table.

        If score = None, then it will be auto calculated.
        If date = None, today's date will be used.
        '''
        row = [
            phonetic_short_vowels,
            phonetic_consonant_blends,
            phonetic_consonant_digraphs,
            transitional_long_vowels,
            transitional_complex_vowels,
            fluent_inflectional_endings,
            fluent_multisyllabic_words_2_syllabes,
            advanced_multisyllabic_words_3_syllabes
        ]

        # calculate potential values

        if not date:
            date = datetime.today().strftime('%Y-%m-%d')

        if not score:
            score = Spelling.calculate_spelling_level(*row)

        # insert into data frame

        row = [ student_id, date ] + row + [ score ]
        row = pd.DataFrame(data= row).T
        row.columns = Spelling.exam_grades.columns
        Spelling.exam_grades = pd.concat([Spelling.exam_grades, row], ignore_index= True)

        # if student is already in table, delete old entry
        Spelling.exam_grades.drop_duplicates(
            subset= ['student_id', 'date'], 
            keep= 'last', 
            inplace= True,
            ignore_index= True
        )

    def get_index_of_student(student_id: int, date: str = None) -> int:
        '''Finds the index in Spelling.exam_grades where the student and date are specified

        If date = None, then finds most recent index.
        '''
        target_index = None

        if not date:
            target_index = Spelling.exam_grades\
            .loc[Spelling.exam_grades.student_id == student_id]\
            .sort_values(by= ['date'], axis= 0)\
            .index[-1]

        else:
            target_index = Spelling.exam_grades.loc[
                (Spelling.exam_grades.student_id == student_id)
                & (Spelling.exam_grades.date == date)
            ].index[0]

        return target_index
    
    def delete_exam_grade(student_id: int, date: str = None) -> None:
        '''Deletes a student's exam grades from the grade book.

        If date = None, then deletes most recent instance.
        If date = "OVERRIDE_ALL", then deletes all instances.
        '''
        target_index = None

        if date == 'OVERRIDE_ALL':
            target_index = Spelling.exam_grades.loc[
                (Spelling.exam_grades.student_id == student_id)
            ].index

        else:
            target_index = Spelling.get_index_of_student(student_id, date)

        Spelling.exam_grades.drop(index= target_index, inplace= True)

    def get_student_response_breakdown(student_id: int, date: str = None) -> dict:
        '''Returns what categories a student got wrong and right on a given exam.

        If date = None, most recent exam is selected.

        Returns dictionary similar to Spelling.stages, except instead of lists, its tuples.
        The first element of the tuple will be correct student response. The second will be
        incorrect.
        '''
        res = dict()
        for key in Spelling.stages:
            res[key] = dict()
            for key_ in Spelling.stages[key]:
                res[key][key_] = None

        target = Spelling.get_index_of_student(student_id, date)
        target = Spelling.exam_grades.iloc[target, range(2, 10)]
        
        for key in res:
            for key_ in res[key]:
                incorrect = set(target.loc[f'{key}_{key_}'])
                correct = list(Spelling.stages[key][key_].difference(incorrect))
                incorrect = list(incorrect)

                res[key][key_] = (correct, incorrect)

        return res

# Reading Class

In [145]:
class Reading:
    '''Reading exam results and resources.
    '''
    exam_grades = pd.DataFrame(columns= ['student_id', 'date', 'reading_level'])

    reading_levels = frozenset([ char for char in string.ascii_uppercase[:26] ])

    def insert_exam_grade(student_id: str, reading_level: str, date: str = None) -> None:
        '''Inserts and updates a students exam info into the exam_grades table.

        If date = None, today's date will be used.
        '''
        if reading_level not in Reading.reading_levels:
            raise Exception(f'Entered reading_level invalid. Expected value in: {Reading.reading_levels}')
        
        if not date:
            date = datetime.today().strftime('%Y-%m-%d')
        
        row = [student_id, date, reading_level]
        row = pd.DataFrame(data= row).T
        row.columns = Reading.exam_grades.columns
        Reading.exam_grades = pd.concat([Reading.exam_grades, row], ignore_index= True)

        # if student is already in table, delete old entry
        Reading.exam_grades.drop_duplicates(
            subset= ['student_id', 'date'], 
            keep= 'last', 
            inplace= True,
            ignore_index= True
        )
    
    def get_index_of_student(student_id: int, date: str = None) -> int:
        '''Finds the index in Reading.exam_grades where the student and date are specified

        If date = None, then finds most recent index.
        '''
        target_index = None

        if not date:
            target_index = Reading.exam_grades\
            .loc[Reading.exam_grades.student_id == student_id]\
            .sort_values(by= ['date'], axis= 0)\
            .index[-1]

        else:
            target_index = Reading.exam_grades.loc[
                (Reading.exam_grades.student_id == student_id)
                & (Reading.exam_grades.date == date)
            ].index[0]

        return target_index
    
    def delete_exam_grade(student_id: int, date: str = None) -> None:
        '''Deletes a student's exam grades from the grade book.

        If date = None, then deletes most recent instance.
        If date = "OVERRIDE_ALL", then deletes all instances.
        '''
        target_index = None

        if date == 'OVERRIDE_ALL':
            target_index = Reading.exam_grades.loc[
                (Reading.exam_grades.student_id == student_id)
            ].index

        else:
            target_index = Reading.get_index_of_student(student_id, date)

        Reading.exam_grades.drop(index= target_index, inplace= True)

    def get_student_response_breakdown(student_id: int, date: str = None) -> dict:
        '''Returns what categories a student got wrong and right on a given exam.

        If date = None, most recent exam is selected.

        Returns str.
        '''
        target = Reading.get_index_of_student(student_id, date)
        target = Reading.exam_grades.iloc[target, 2].iloc[0]

        return target
    

# Demo

In [84]:
from faker import Faker
import random

Faker.seed(420)

## Populating Students

In [86]:
fake = Faker()
first_names = [ fake.unique.first_name() for i in range(100) ]
last_names = [ fake.unique.last_name() for i in range(100) ]
genders = random.choices(['male', 'female', 'nonbinary', 'agender', 'gamer', 'freddyFazzbear'], k= 100)
ages = random.choices([34, 33, 32], k= 100)
ids = range(200, 200 + 95 + 1)

Observe how everything specified is entered into our database as explicitly given

In [87]:
for i in range(95):
    Students.insert_student(
        str(first_names[i]),
        str(last_names[i]),
        genders[i],
        ages[i],
        ids[i]
    )

Students.students

,id,first_name,last_name,gender,age
0,200,anthony,griffin,nonbinary,32
1,201,kimberly,sosa,female,34
2,202,joseph,coleman,nonbinary,33
3,203,cody,barajas,freddyfazzbear,32
4,204,elizabeth,mullen,male,32
...,...,...,...,...,...
90,290,stacey,sanchez,freddyfazzbear,34
91,291,adrian,adams,agender,32
92,292,angelica,lynch,freddyfazzbear,34
93,293,holly,long,freddyfazzbear,32


Now observe that we don't have to worry about specifying the id. The program figures that out for us. This is a super trivial example. Look at the code for more info.

In [88]:
for i in range(95, 100):
    Students.insert_student(
        str(first_names[i]),
        str(last_names[i]),
        genders[i],
        ages[i]
    )

Students.students

,id,first_name,last_name,gender,age
0,200,anthony,griffin,nonbinary,32
1,201,kimberly,sosa,female,34
2,202,joseph,coleman,nonbinary,33
3,203,cody,barajas,freddyfazzbear,32
4,204,elizabeth,mullen,male,32
...,...,...,...,...,...
95,1,melinda,owens,agender,33
96,2,amber,donovan,agender,32
97,3,tyler,brady,agender,32
98,4,ashlee,jones,freddyfazzbear,34


delete a student of your choosing

you can use student id ....

In [89]:
Students.delete_student(
    student_id= 201
)

Students.students

,id,first_name,last_name,gender,age
0,200,anthony,griffin,nonbinary,32
2,202,joseph,coleman,nonbinary,33
3,203,cody,barajas,freddyfazzbear,32
4,204,elizabeth,mullen,male,32
5,205,diana,guzman,male,33
...,...,...,...,...,...
95,1,melinda,owens,agender,33
96,2,amber,donovan,agender,32
97,3,tyler,brady,agender,32
98,4,ashlee,jones,freddyfazzbear,34


... or you can use first name, last name

In [90]:
Students.delete_student(  
    first_name= 'joseph', last_name= 'coleman'
)

Students.students

,id,first_name,last_name,gender,age
0,200,anthony,griffin,nonbinary,32
3,203,cody,barajas,freddyfazzbear,32
4,204,elizabeth,mullen,male,32
5,205,diana,guzman,male,33
6,206,mark,goodwin,gamer,33
...,...,...,...,...,...
95,1,melinda,owens,agender,33
96,2,amber,donovan,agender,32
97,3,tyler,brady,agender,32
98,4,ashlee,jones,freddyfazzbear,34


we can also update an existing student. Set any params below as anything you want, BUT the student_id HAS to exist for this to work.

In [91]:
Students.insert_student(   # note that these names are rnaodmly generated and may not work for you because I was too lazy to set seed
    first_name= 'codyNEW',  # I changed first name
    last_name= 'barajas',  # keeping it original
    gender= 'nonbinary',  # changed gender
    age= 32,  # keeping it original
    student_id= 203  # MUST already exist in the table in order to update an entry
)

Students.students

,id,first_name,last_name,gender,age
0,200,anthony,griffin,nonbinary,32
1,204,elizabeth,mullen,male,32
2,205,diana,guzman,male,33
3,206,mark,goodwin,gamer,33
4,207,yesenia,peterson,agender,32
...,...,...,...,...,...
93,2,amber,donovan,agender,32
94,3,tyler,brady,agender,32
95,4,ashlee,jones,freddyfazzbear,34
96,5,margaret,watson,male,32


## Populating Comprehension

In [97]:
ids = range(1, 101)
types = ['fictional'] * 50 + ['informational'] * 50
incorrects = random.choices(list(Comprehension.fictional), k= 50) + random.choices(list(Comprehension.informational), k= 50)  # everyone will only get 1 wrong
dates = [fake.date() for i in range(95)]

Observe how everything specified is entered into our database as explicitly given

In [130]:
for i in range(95):
    Comprehension.insert_exam_grade(
        student_id= ids[i],
        type_= types[i],
        incorrect= [incorrects[i]],
        date= dates[i],
        score= 7
    )

Comprehension.exam_grades

,student_id,type,date,incorrect,score
0,1,fictional,1992-02-02,[go],7
1,2,fictional,1987-03-12,[ca],7
2,3,fictional,1997-10-27,[e],7
3,4,fictional,2016-03-09,[v],7
4,5,fictional,2022-08-20,[l],7
...,...,...,...,...,...
90,91,informational,2009-09-30,[e],7
91,92,informational,2015-09-20,[tf],7
92,93,informational,1991-05-08,[e],7
93,94,informational,1993-08-01,[mi],7


Now observe that we don't have to worry about specifying the date or score

In [131]:
Comprehension.insert_exam_grade(
        student_id= 1,
        type_= 'fictional',
        incorrect= [incorrects[0], incorrects[1]],
    )

Comprehension.exam_grades

,student_id,type,date,incorrect,score
0,1,fictional,1992-02-02,[go],7
1,2,fictional,1987-03-12,[ca],7
2,3,fictional,1997-10-27,[e],7
3,4,fictional,2016-03-09,[v],7
4,5,fictional,2022-08-20,[l],7
...,...,...,...,...,...
91,92,informational,2015-09-20,[tf],7
92,93,informational,1991-05-08,[e],7
93,94,informational,1993-08-01,[mi],7
94,95,informational,1987-06-25,[v],7


- Just like in student, you can update a student's exam grade by using the insert function. However, you have to specify student_id and date
- Similarly, you can delete a single student record at a time, their most recent record by specifying date= None, or all their records via date= 'OVERRIDE_ALL'

In [132]:
Comprehension.delete_exam_grade(
    student_id= 1
)

Comprehension.exam_grades

,student_id,type,date,incorrect,score
0,1,fictional,1992-02-02,[go],7
1,2,fictional,1987-03-12,[ca],7
2,3,fictional,1997-10-27,[e],7
3,4,fictional,2016-03-09,[v],7
4,5,fictional,2022-08-20,[l],7
...,...,...,...,...,...
90,91,informational,2009-09-30,[e],7
91,92,informational,2015-09-20,[tf],7
92,93,informational,1991-05-08,[e],7
93,94,informational,1993-08-01,[mi],7


In [133]:
Comprehension.insert_exam_grade(
        student_id= 1,
        type_= 'fictional',
        incorrect= [incorrects[0], incorrects[1]],
    )

Comprehension.delete_exam_grade(
    student_id= 1,
    date= 'OVERRIDE_ALL'
)

Comprehension.exam_grades

,student_id,type,date,incorrect,score
1,2,fictional,1987-03-12,[ca],7
2,3,fictional,1997-10-27,[e],7
3,4,fictional,2016-03-09,[v],7
4,5,fictional,2022-08-20,[l],7
5,6,fictional,2013-08-19,[go],7
...,...,...,...,...,...
90,91,informational,2009-09-30,[e],7
91,92,informational,2015-09-20,[tf],7
92,93,informational,1991-05-08,[e],7
93,94,informational,1993-08-01,[mi],7


you can also see a grade report for a specifc students exam. The first elem in tuple is everything the student got correct. The second is everything they got wrong.

In [134]:
Comprehension.get_student_response_breakdown(
    student_id= 2
)

(['mi', 'v', 'tf', 'ar', 'kd', 'i', 'l'], ['e'])

## Populating Spelling

In [160]:
ids = range(1, 101)
p1 = random.choices(list(Spelling.stages['phonetic']['short_vowels']), k= 100)
p2 = random.choices(list(Spelling.stages['phonetic']['consonant_blends']), k= 100)
p3 = random.choices(list(Spelling.stages['phonetic']['consonant_digraphs']), k= 100)
t1 = random.choices(list(Spelling.stages['transitional']['long_vowels']), k= 100)
t2 = random.choices(list(Spelling.stages['transitional']['complex_vowels']), k= 100)
f1 = random.choices(list(Spelling.stages['fluent']['inflectional_endings']), k= 100)
f2 = random.choices(list(Spelling.stages['fluent']['multisyllabic_words_2_syllabes']), k= 100)
a1 = random.choices(list(Spelling.stages['advanced']['multisyllabic_words_3_syllabes']), k= 100)

Like with the previous two classes, we can manually specify eveyrthing. Or we can leave date and score blank. When score is blank, we compute the spelling level of the student. I don't know how to do that yet, so there's like nothing going on with that in the background tho.

In [161]:
for i in range(100):
    Spelling.insert_exam_grade(  # this is super dumb dummy data. Its basically saying that the student only got one thing wrong from each category
        student_id= ids[i],
        phonetic_short_vowels= [p1[i]],
        phonetic_consonant_blends= [p2[i]],
        phonetic_consonant_digraphs= [p3[i]],
        transitional_long_vowels= [t1[i]],
        transitional_complex_vowels= [t2[i]],
        fluent_inflectional_endings= [f1[i]],
        fluent_multisyllabic_words_2_syllabes= [f2[i]],
        advanced_multisyllabic_words_3_syllabes= [a1[i]]
    )

Spelling.exam_grades

,student_id,date,phonetic_short_vowels,phonetic_consonant_blends,phonetic_consonant_digraphs,transitional_long_vowels,transitional_complex_vowels,fluent_inflectional_endings,fluent_multisyllabic_words_2_syllabes,advanced_multisyllabic_words_3_syllabes,score
0,1,2023-10-25,[o],[-st],[th],[ea],[aw],[-edt /t/],[-ful (suffix)],[-ate],phonetic
1,2,2023-10-25,[u],[-nk],[sh],[o_e],[oo],[ed],[-ful (suffix)],[-ure],phonetic
2,3,2023-10-25,[u],[-st],[sh],[ea],[oo],[-edt /t/],[-ness (suffix)],[-ure],phonetic
3,4,2023-10-25,[e],[tr],[th],[o_e],[ou],[ed],[-tion (suffix)],[-ous],phonetic
4,5,2023-10-25,[o],[-nk],[sh],[ea],[ur],[-ing (double consonant)],[-ness (suffix)],[-ity],phonetic
...,...,...,...,...,...,...,...,...,...,...,...
95,96,2023-10-25,[u],[-nk],[th],[oa],[ur],[-ing (spelling change)],[-ful (suffix)],[-ate],phonetic
96,97,2023-10-25,[u],[sl],[sh],[o_e],[oo],[ed],[-tion (suffix)],[-ate],phonetic
97,98,2023-10-25,[e],[cl],[ch],[ea],[or],[ed],[-tion (suffix)],[-ous],phonetic
98,99,2023-10-25,[u],[-nk],[th],[o_e],[ou],[-s (spelling change)],[mar (r-controlled syllable)],[-able],phonetic


- You can update and delete things in the same way as Comprehension

The grade report is detailed per phenetic stage sub-part. Tuple first item is what the student got correct in that section, second item is what they got wrong.

In [162]:
Spelling.get_student_response_breakdown(5)

{'phonetic': {'short_vowels': (['u', 'a', 'e'], ['o']),
  'consonant_blends': (['fr',
    'dr',
    'cl',
    'fl',
    'sl',
    'spr',
    '-st',
    'tr',
    'gr'],
   ['-nk']),
  'consonant_digraphs': (['th', 'ck', 'ch'], ['sh'])},
 'transitional': {'long_vowels': (['igh', 'o_e', 'oa', 'ai'], ['ea']),
  'complex_vowels': (['or', 'ou', 'oo', 'oi', 'aw', 'ir'], ['ur'])},
 'fluent': {'inflectional_endings': (['-edt /t/',
    'ed',
    '-s (spelling change)',
    '-ing (spelling change)'],
   ['-ing (double consonant)']),
  'multisyllabic_words_2_syllabes': (['-ful (suffix)',
    'ble (end syllable)',
    '-tion (suffix)',
    'mar (r-controlled syllable)'],
   ['-ness (suffix)'])},
 'advanced': {'multisyllabic_words_3_syllabes': (['-able',
    '-ous',
    '-ate',
    '-ure'],
   ['-ity'])}}

## Populating Reading

- reading follows all the same patterns we've previously seen